In [1]:
from midir import w1w2_condition
from tqdm import tqdm

import os
import numpy as np
from collections import OrderedDict as od
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as tkr
from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D
import csv
from matplotlib import path

try:
    from astropy.convolution import Gaussian2DKernel, convolve
    astro_smooth = True
except ImportError as IE:
    astro_smooth = False

from astropy.io import fits

from astropy.coordinates import match_coordinates_sky as coords
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles

#from astropy.units import cds
#cds.enable()

from astropy.cosmology import LambdaCDM, FlatLambdaCDM

from astropy.io import ascii
from astropy.io.ascii import masked
from astropy.table import Table

from scipy.interpolate import interp1d
from scipy.stats import gaussian_kde
from numpy import polyfit
from astropy.time import Time

#import uncertainties as unc  
#import uncertainties.unumpy as unp
import time

import kcorrect.response

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 22})

SAVE_FILEPATH = os.path.join(os.path.curdir, 'WISE variations/Final Plots 2.2.0/')
IMPORT_FILEPATH = os.path.join(os.path.curdir, 'WISE variations/')

In [2]:
manga_file = fits.open(IMPORT_FILEPATH + "mnsa-0.3.0.fits")
manga_hdu = manga_file[1]
manga_data = manga_hdu.data

pipe3d_file = fits.open(IMPORT_FILEPATH + 'SDSS17Pipe3D_v3_1_1.fits')
pipe3d_hdu = pipe3d_file[1]
pipe3d_data = pipe3d_hdu.data

wise_file = fits.open(IMPORT_FILEPATH + "manga-wise-variable-0.3.0.fits")
wise_hdu = wise_file[1]
whdu = wise_hdu.data

jiyan_file = fits.open(IMPORT_FILEPATH + "jiyan-agn-0.3.2.fits")
jiyan_hdu = jiyan_file[1]
jhdu = jiyan_hdu.data

summary = manga_file[1].data
sps2 = manga_file[2].data

has_pipe3d = summary['ipipe3d '] != -1

sps2_matched = sps2[has_pipe3d]
summary_matched = summary[has_pipe3d]
wise_matched = whdu[has_pipe3d]

pipe3d_matched = pipe3d_data[summary_matched['ipipe3d ']]

In [3]:
x1 = wise_matched['expected_W2_var_all_mags']
y1 = wise_matched['observed_W2_var']
plateifu = wise_matched['plateifu']

w1 =  sps2_matched['maggies'][:, 5]
w2 =  sps2_matched['maggies'][:, 6]
w1_app = -2.5 * np.log10(w1)
w2_app = -2.5 * np.log10(w2)
w12 = w1_app-w2_app

mask = (wise_matched['epoch_flag'] == 0)
plateifu, x1, y1, w1_app, w2_app = plateifu[mask], x1[mask], y1[mask], w1_app[mask], w2_app[mask]
mask2 = (plateifu != '8239-3701')
plateifu, x1, y1, w1_app, w2_app = plateifu[mask2], x1[mask2], y1[mask2], w1_app[mask2], w2_app[mask2]

#mask = ((x1!=-9999.) & (y1!=-9999.)) #(np.isfinite(w1_app) & np.isfinite(w2_app) &

#x1, y1, plateifu, w1_app, w2_app = x1[mask], y1[mask], plateifu[mask], w1_app[mask], w2_app[mask]

C:\Users\paiaa\AppData\Local\Temp\ipykernel_20136\1597797605.py:7: RuntimeWarning: divide by zero encountered in log10
  w1_app = -2.5 * np.log10(w1)
C:\Users\paiaa\AppData\Local\Temp\ipykernel_20136\1597797605.py:8: RuntimeWarning: divide by zero encountered in log10
  w2_app = -2.5 * np.log10(w2)
C:\Users\paiaa\AppData\Local\Temp\ipykernel_20136\1597797605.py:8: RuntimeWarning: invalid value encountered in log10
  w2_app = -2.5 * np.log10(w2)
C:\Users\paiaa\AppData\Local\Temp\ipykernel_20136\1597797605.py:9: RuntimeWarning: invalid value encountered in subtract
  w12 = w1_app-w2_app


In [4]:
#variable = ((np.log10(y1)>-2.5) & (np.log10(y1)>0.92*np.log10(x1)+0.43) & (np.log10(x1)<-2))
#variable = ((np.log10(x1)<-2.5) & (np.log10(y1)>-2.3) & (np.log10(y1)>=2.25*np.log10(x1)+4.09) & (np.log10(y1)>=0.6*np.log10(x1)-0.38))
variable = ((np.log10(x1)<-2.5) & (np.log10(y1)>-2.3) & (np.log10(y1)>np.log10(x1)+0.7))

pifuv, w1v, w2v = plateifu[variable], w1_app[variable], w2_app[variable]

pifu, imanga, ipifuv = np.intersect1d(summary_matched['plateifu'], pifuv, return_indices=True)
pifu, ipifu, ipifuv = np.intersect1d(wise_matched['plateifu'], pifuv, return_indices=True)

In [5]:
pifuv.shape

(170,)

In [5]:
wise_variable = wise_matched[ipifu]
summary_variable = summary_matched[imanga]
sps2_variable = sps2_matched[imanga]

In [10]:
catalog = Table()
catalog['Plateifu'] = wise_variable['plateifu']
catalog['RA'] = np.around(summary_variable['objra'], 4)
catalog['Dec'] = np.around(summary_variable['objdec'], 4)
catalog['W1'] =  np.around(w1v, 2)
catalog['W2'] = np.around(w2v, 2)
catalog[r'$\log_{10}\varsigma^2$'] = np.around(np.log10(wise_variable['expected_W2_var_all_mags']), 3)
catalog[r'$\text{Var}(W2)$'] = np.around(np.log10(wise_variable['observed_W2_var']), 3)
catalog.sort(['RA'])
#catalog.write(SAVE_FILEPATH+ 'variability_catalog.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)


In [7]:
catalog[:45].write(SAVE_FILEPATH+'var_cat1.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)
catalog[45:90].write(SAVE_FILEPATH+'var_cat2.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)
catalog[90:135].write(SAVE_FILEPATH+'var_cat3.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)
catalog[135:].write(SAVE_FILEPATH+'var_cat4.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)
#catalog[180:].write(SAVE_FILEPATH+'var_cat5.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)
#catalog[125:170].write(SAVE_FILEPATH+'var_cat6.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)
#catalog[170:].write(SAVE_FILEPATH+'var_cat7.tex', format='latex', latexdict=ascii.latex.latexdicts['AA'], overwrite=True)

In [8]:
catalog.sort([r'$\text{Var}(W2)$'], reverse=True)
catalog.write(SAVE_FILEPATH+ 'variability_catalog', format='ascii', overwrite=True)


In [9]:
catalog

Plateifu,RA,Dec,W1,W2,$\log_{10}\varsigma^2$,$\text{Var}(W2)$
str12,float64,float64,float32,float32,float64,float64
9870-1902,233.1651,44.5342,17.09,17.53,-3.062,-0.3
8145-6102,116.5535,26.923,15.58,16.05,-2.585,-0.823
8710-6102,117.9662,49.8143,13.91,14.27,-3.913,-0.84
10223-3701,31.5666,-0.2914,13.08,13.25,-3.854,-0.845
11011-1902,218.7186,48.6619,15.42,15.63,-4.131,-0.897
9000-1901,171.4007,54.3826,13.79,13.75,-4.575,-0.898
8341-12704,189.2133,45.6512,17.85,18.23,-3.781,-0.997
8561-3704,241.3254,52.1202,16.93,17.32,-3.656,-1.084
8940-12702,120.0874,26.6135,12.66,12.66,-4.137,-1.085


In [26]:
p = np.array([])
for pifu in catalog['Plateifu']:
    p = np.append(p, 'M'+ pifu.split('-')[0]+pifu.split('-')[1])

In [31]:
var_table = Table()
var_table['ID'] = catalog['Plateifu']
var_table['RA'] = catalog['RA']
var_table['Dec'] = catalog['Dec']

In [33]:
var_table[:100].write(SAVE_FILEPATH+ 'midir_var_objs_1.txt', format='ascii', overwrite=True)
var_table[100:].write(SAVE_FILEPATH+ 'midir_var_objs_2.txt', format='ascii', overwrite=True)